In [119]:
from itertools import permutations
from pathlib import Path

import numpy as np

from score_strokes import strokeErrorMatrix
from xmlparse import getXmlScore, loadGeometryBases, loadRef, minXml

## Dynamic Programming Algorithm

In [120]:
"""
Currently the function is broken, as it does not return scores correctly with multiple gene files in the directory. Can still be tested for one though.
Implementation and design of a dynamic programming algorithm are probably incorrect as well, requires heavy testing and more learning on my part.
At the moment the algorithm is incompatible with Holiday's API because it requires scoring multiple stroke orders (not O(n)).
"""

# Testing a dynamic programming algorithm
def dynamic(ref_char, ref_data, char_data):
    ref, p_ref, _ = ref_data
    g_data, _, base_data, stroke_sets, _, f_names = char_data
    stroke_priority = permutations(range(0, len(ref)))
    heuristic_scores = []
    for (geometry_length, bases, stroke_set, f_name) in zip(g_data, base_data, stroke_sets, f_names):
        stroke_maps = []
        strokes, p_strokes = geometry_length
        error_maps = strokeErrorMatrix(strokes, ref, p_strokes, p_ref)
        # Find candidate stroke orders
        for priority in stroke_priority:
            stroke_map = np.full(len(strokes), -1)
            for i in priority:
                smallerror = np.argmin(error_maps[i]) # retrieve index of smallest error for current archetype stroke
                while(stroke_map[smallerror]!=-1):
                    # change small error so that we do not repeat over indexes that are already taken
                    # just keeps repeating until we land on an index that doesn't already have a value in its place
                    error_maps[i][smallerror] = 10000
                    smallerror = np.argmin(error_maps[i])
                stroke_map[smallerror] = i
            if not any(np.array_equal(stroke_map, m) for m in stroke_maps):
                stroke_maps.append(stroke_map)
        # Retrieve scores for each candidate stroke order
        for s in stroke_maps:
            heuristic_xml = minXml(ref_char, bases, stroke_set, np.array(s)+1)
            heuristic_score = getXmlScore(heuristic_xml)
            heuristic_scores.append(heuristic_score)
    return heuristic_scores

ref_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/Reference' # archetype directory
data_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/NewGenes' # gene directory
ref_char = "6709"

ref_data = loadRef(ref_char, ref_dir)
char_data = loadGeometryBases(data_dir, ref_data[2])

dynamic(ref_char, ref_data, char_data)

2024-06-05T22:26:49.910876Z [INFO ] Loaded genome  containing 1 genes - trial set to 0
2024-06-05T22:26:49.910945Z [INFO ] TRIAL 0: Fitness is 0.001342939948137


[0.0001031825318209632]

## Heuristic Comparison Function

In [121]:
"""
Function to compare two heuristic algorithms' accuracy and performance.
"""
def compareHeuristic(algo1, algo2):
    return